In [1]:
from pymongo import MongoClient
import pandas as pd
import time



# Connect to db

In [2]:
client = MongoClient('localhost', 27017)
db = client.gfif

In [3]:
db.eval("db.stats()")

{'avgObjSize': 945.9603330269452,
 'collections': 4,
 'dataFileVersion': {'major': 4, 'minor': 5},
 'dataSize': 92469372580.0,
 'db': 'gfif',
 'fileSize': 128720044032.0,
 'indexSize': 28523660256.0,
 'indexes': 5,
 'nsSizeMB': 16,
 'numExtents': 66,
 'objects': 97751850,
 'ok': 1.0,
 'storageSize': 94625086832.0}

# Query 0
Find a register(document for mongo) and show the content

In [4]:
start = time.time()
reg=db.papers.find({"doi" : "10.7705/biomedica.v37i3.4058"})
end = time.time()
print("query time = %f sec"%(end - start))
regdict=reg.next()
print(regdict["is_oa"])
print(regdict["oa_locations"][0]["url"])
regdict

query time = 0.000135 sec
True
https://www.revistabiomedica.org/index.php/biomedica/article/download/4058/3623


{'_id': ObjectId('5bce71952bb1ea77e3dd02ad'),
 'best_oa_location': {'evidence': 'open (via free pdf)',
  'host_type': 'publisher',
  'is_best': True,
  'license': None,
  'pmh_id': None,
  'updated': '2018-02-15T05:20:07.183468',
  'url': 'https://www.revistabiomedica.org/index.php/biomedica/article/download/4058/3623',
  'url_for_landing_page': 'https://doi.org/10.7705/biomedica.v37i3.4058',
  'url_for_pdf': 'https://www.revistabiomedica.org/index.php/biomedica/article/download/4058/3623',
  'version': 'publishedVersion'},
 'data_standard': 2,
 'doi': '10.7705/biomedica.v37i3.4058',
 'doi_url': 'https://doi.org/10.7705/biomedica.v37i3.4058',
 'genre': 'journal-article',
 'is_oa': True,
 'journal_is_in_doaj': False,
 'journal_is_oa': False,
 'journal_issns': '0120-4157,0120-4157',
 'journal_name': 'Biomédica',
 'oa_locations': [{'evidence': 'open (via free pdf)',
   'host_type': 'publisher',
   'is_best': True,
   'license': None,
   'pmh_id': None,
   'updated': '2018-02-15T05:20:07.1

# Query 1
grep -E '"is_oa":\s*true' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl  >  oa.json

grep -E '"journal_is_oa":\s*true' oa.json > oagreen.json

In [5]:
start = time.time()
oagreen = db.papers.find({"is_oa":True,"journal_is_oa":True})
end = time.time()
print("query time = %f sec"%(end - start))

query time = 0.000134 sec


In [6]:
#printing some results
n=6
for i in range(n):
    print(oagreen.next()["doi"])

10.1371/journal.pone.0144703.g008
10.1371/journal.pone.0073259.s001
10.1371/journal.pgen.0020184.g005
10.1371/journal.pone.0107486.s003
10.1371/journal.pone.0130331.t001
10.4081/ni.2016.6553


# Iteration with cursor
* Prodemos iterar con el metodo next() del objecto que retorne el find
* para redgresar el cursor al principio rewind()

ver más en http://api.mongodb.com/python/current/api/pymongo/cursor.html


In [13]:
#for i in range(2):
#    current=oagreen.next()
#    #do something more

# Query 2
grep -E '"is_oa":\s*true' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl  >  oa.json

grep -E '"journal_is_oa":\s*false' oa.json > oatmp.json

In [8]:
start = time.time()
oatmp = db.papers.find({"is_oa":True,"journal_is_oa":False})
end = time.time()
print("query time = %f sec"%(end - start))

query time = 0.000202 sec


In [9]:
#printing some results
n=6
for i in range(n):
    print(oatmp.next()["doi"])

10.1097/00007890-201007272-00675
10.6118/jmm.2017.23.2.135
10.1002/j.2333-8504.2008.tb02147.x
10.1351/goldbook.s05545
10.1093/ndt/gfq052
10.1016/j.ahj.2015.02.019


In [10]:
dois=df=pd.read_csv('test.csv')

In [31]:
results=db.papers.find({"doi":{'$in':dois["DOI"].values.tolist()}})

In [32]:
for i in results:
    print(i)


{'publisher': 'FapUNIFESP (SciELO)', 'oa_locations': [{'url': 'http://www.scielo.br/pdf/rsbmt/v44n3/aop30-11.pdf', 'is_best': True, 'updated': '2017-09-10T00:43:04.762391', 'license': 'cc-by-nc', 'evidence': 'open (via page says license)', 'host_type': 'publisher', 'url_for_landing_page': 'http://doi.org/10.1590/s0037-86822011005000030', 'url_for_pdf': 'http://www.scielo.br/pdf/rsbmt/v44n3/aop30-11.pdf', 'pmh_id': None, 'version': 'publishedVersion'}, {'url': 'https://doi.org/10.1590/s0037-86822011005000030', 'is_best': False, 'updated': '2018-06-16T20:44:10.724134', 'license': 'cc-by', 'evidence': 'oa journal (via doaj)', 'host_type': 'publisher', 'url_for_landing_page': 'https://doi.org/10.1590/s0037-86822011005000030', 'url_for_pdf': None, 'pmh_id': None, 'version': 'publishedVersion'}], 'best_oa_location': {'url': 'http://www.scielo.br/pdf/rsbmt/v44n3/aop30-11.pdf', 'is_best': True, 'updated': '2017-09-10T00:43:04.762391', 'license': 'cc-by-nc', 'evidence': 'open (via page says lic

In [33]:
results.rewind()
df = pd.DataFrame(list(results))

In [34]:
df

,_id,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors
0,5bce724a2bb1ea77e3f73d2b,{'url': 'http://www.scielo.br/pdf/rsbmt/v44n3/...,2,10.1590/s0037-86822011005000030,https://doi.org/10.1590/s0037-86822011005000030,journal-article,True,True,True,0037-8682,Revista da Sociedade Brasileira de Medicina Tr...,[{'url': 'http://www.scielo.br/pdf/rsbmt/v44n3...,2011-06-01,FapUNIFESP (SciELO),"Retardo no crescimento intrauterino, baixo pes...",2018-06-16T20:44:10.726101,2011,"[{'given': 'Alberto', 'family': 'Tobón-Castaño..."
1,5bce66122bb1ea77e33550d4,{'url': 'http://www.scielo.br/pdf/rbce/v34n1/v...,2,10.1590/s0101-32892012000100010,https://doi.org/10.1590/s0101-32892012000100010,journal-article,True,True,True,0101-3289,Revista Brasileira de Ciências do Esporte,[{'url': 'http://www.scielo.br/pdf/rbce/v34n1/...,2012-03-01,FapUNIFESP (SciELO),Un análisis parcial a la producción académica ...,2018-06-14T16:57:39.583210,2012,"[{'given': 'León Jaime Urrego', 'family': 'Duq..."
2,5bce6aab2bb1ea77e3ded0d9,{'url': 'http://www.scielo.br/pdf/his/v30n1/v3...,2,10.1590/s0101-90742011000100014,https://doi.org/10.1590/s0101-90742011000100014,journal-article,True,True,True,0101-9074,História (São Paulo),[{'url': 'http://www.scielo.br/pdf/his/v30n1/v...,2011-06-01,FapUNIFESP (SciELO),Los inicios del astillero de la Habana en el s...,2018-06-16T17:57:36.403923,2011,"[{'given': 'José Manuel', 'family': 'Serrano A..."
3,5bce4da22bb1ea77e3feafc9,{'url': 'https://doi.org/10.1590/s0104-4060201...,1,10.1590/s0104-40602012000200016,https://doi.org/10.1590/s0104-40602012000200016,journal-article,True,True,True,0104-4060,Educar em Revista,[{'url': 'https://doi.org/10.1590/s0104-406020...,2012-06-01,FapUNIFESP (SciELO),Formación y gusto por la escritura y la lectur...,2018-06-20T15:05:02.422033,2012,"[{'given': 'Carmen Emilia', 'family': 'García ..."
4,5bce6d242bb1ea77e33a960d,{'url': 'https://doi.org/10.1590/s1414-462x201...,1,10.1590/s1414-462x2012000400002,https://doi.org/10.1590/s1414-462x2012000400002,journal-article,True,False,True,1414-462X,Cadernos Saúde Coletiva,[{'url': 'https://doi.org/10.1590/s1414-462x20...,2012-01-01,FapUNIFESP (SciELO),Programa de prevención temprana de conductas d...,2018-06-16T19:35:46.032637,2012,"[{'given': 'Alexandra', 'family': 'Restrepo He..."
5,5bce73fd2bb1ea77e335d86f,{'url': 'http://www.scielo.br/pdf/ciedu/v18n4/...,2,10.1590/s1516-73132012000400002,https://doi.org/10.1590/s1516-73132012000400002,journal-article,True,True,True,1516-7313,Ciência & Educação (Bauru),[{'url': 'http://www.scielo.br/pdf/ciedu/v18n4...,2012-01-01,FapUNIFESP (SciELO),La medición de la capacidad de resolución de p...,2018-06-18T20:51:34.539795,2012,"[{'given': 'José Joaquín', 'family': 'García G..."
6,5bce635f2bb1ea77e3e1034d,{'url': 'https://scielo.conicyt.cl/pdf/udecada...,2,10.4067/s0718-22362011000200003,https://doi.org/10.4067/s0718-22362011000200003,journal-article,True,False,True,0718-2236,Ultima década,[{'url': 'https://scielo.conicyt.cl/pdf/udecad...,2011-12-01,SciELO Comision Nacional de Investigacion Cien...,PRÁCTICAS Y FENÓMENOS EMERGENTES EN LA JUVENTU...,2018-06-18T13:45:23.851519,2011,"[{'given': 'Manuel E', 'family': 'López García'}]"
7,5bce6f392bb1ea77e386139d,{'url': 'https://scielo.conicyt.cl/pdf/ingenia...,2,10.4067/s0718-33052011000300009,https://doi.org/10.4067/s0718-33052011000300009,journal-article,True,True,True,0718-3305,Ingeniare. Revista chilena de ingeniería,[{'url': 'https://scielo.conicyt.cl/pdf/ingeni...,2011-12-01,SciELO Comision Nacional de Investigacion Cien...,Identificación y caracterización de mudas de t...,2018-06-15T18:05:32.348440,2011,"[{'given': 'Jorge', 'family': 'Pérez Rave'}, {..."
8,5bce75322bb1ea77e3639ab9,{'url': 'https://scielo.conicyt.cl/pdf/ijodont...,2,10.4067/s0718-381x2012000200015,https://doi.org/10.4067/s0718-381x2012000200015,journal-article,True,False,True,0